<div style="text-align: right"><b> Ana Valentina López Chacón </b></div>
<div style="text-align: right"><b> Traducción Automática, 2024 - 2025 </b></div>

# **Python a Ingles: CodeT5 Baseline**

In [1]:
!pip install -q git+https://github.com/openai/whisper.git
!pip install -q datasets jiwer tqdm pandas
!pip install -q torch torchvision torchaudio
!pip install -q evaluate
!pip install -q transformers==4.47.0
!pip install -q accelerate==0.27.0
!pip install -q peft
!pip install -q bitsandbytes==0.45.0
!pip install -q sacrebleu
!pip install -q unbabel-comet
!pip install -q rouge_score

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 20.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 14.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 9.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 16.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 13.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 72.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 19.5 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [2]:
from datasets import load_dataset

dataset = load_dataset("code_x_glue_ct_code_to_text", "python")

dataset

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/26.7k [00:00<?, ?B/s]

train-00000-of-00002.parquet:   0%|          | 0.00/144M [00:00<?, ?B/s]

train-00001-of-00002.parquet:   0%|          | 0.00/147M [00:00<?, ?B/s]

validation-00000-of-00001.parquet:   0%|          | 0.00/16.7M [00:00<?, ?B/s]

test-00000-of-00001.parquet:   0%|          | 0.00/18.0M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/251820 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/13914 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/14918 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['id', 'repo', 'path', 'func_name', 'original_string', 'language', 'code', 'code_tokens', 'docstring', 'docstring_tokens', 'sha', 'url'],
        num_rows: 251820
    })
    validation: Dataset({
        features: ['id', 'repo', 'path', 'func_name', 'original_string', 'language', 'code', 'code_tokens', 'docstring', 'docstring_tokens', 'sha', 'url'],
        num_rows: 13914
    })
    test: Dataset({
        features: ['id', 'repo', 'path', 'func_name', 'original_string', 'language', 'code', 'code_tokens', 'docstring', 'docstring_tokens', 'sha', 'url'],
        num_rows: 14918
    })
})

In [3]:
dataset['train'].features

{'id': Value(dtype='int32', id=None),
 'repo': Value(dtype='string', id=None),
 'path': Value(dtype='string', id=None),
 'func_name': Value(dtype='string', id=None),
 'original_string': Value(dtype='string', id=None),
 'language': Value(dtype='string', id=None),
 'code': Value(dtype='string', id=None),
 'code_tokens': Sequence(feature=Value(dtype='string', id=None), length=-1, id=None),
 'docstring': Value(dtype='string', id=None),
 'docstring_tokens': Sequence(feature=Value(dtype='string', id=None), length=-1, id=None),
 'sha': Value(dtype='string', id=None),
 'url': Value(dtype='string', id=None)}

In [4]:
print(len(dataset['train']))
print(len(dataset['validation']))
print(len(dataset['test']))

251820
13914
14918


In [5]:
seed = 42

train_dataset = dataset['train'].shuffle(seed=seed).select(range(200))
validation_dataset = dataset['validation'].shuffle(seed=seed).select(range(1000))
test_dataset = dataset['test'].shuffle(seed=seed).select(range(1000))

print("Training dataset size:", len(train_dataset))
print("Validation dataset size:", len(validation_dataset))
print("Test dataset size:", len(test_dataset))

Training dataset size: 200
Validation dataset size: 1000
Test dataset size: 1000


In [6]:
train_dataset[0]['code']

'def metapolicy(request, permitted, domains=None):\n    """\n    Serves a cross-domain policy which can allow other policies\n    to exist on the same domain.\n\n    Note that this view, if used, must be the master policy for the\n    domain, and so must be served from the URL ``/crossdomain.xml`` on\n    the domain: setting metapolicy information in other policy files\n    is forbidden by the cross-domain policy specification.\n\n    **Required arguments:**\n\n    ``permitted``\n        A string indicating the extent to which other policies are\n        permitted. A set of constants is available in\n        ``flashpolicies.policies``, defining acceptable values for\n        this argument.\n\n    **Optional arguments:**\n\n    ``domains``\n        A list of domains from which to allow access. Each value may\n        be either a domain name (e.g., ``example.com``) or a wildcard\n        (e.g., ``*.example.com``). Due to serious potential security\n        issues, it is strongly recommen

In [7]:
train_dataset[0]["docstring"]

'Serves a cross-domain policy which can allow other policies\n    to exist on the same domain.\n\n    Note that this view, if used, must be the master policy for the\n    domain, and so must be served from the URL ``/crossdomain.xml`` on\n    the domain: setting metapolicy information in other policy files\n    is forbidden by the cross-domain policy specification.\n\n    **Required arguments:**\n\n    ``permitted``\n        A string indicating the extent to which other policies are\n        permitted. A set of constants is available in\n        ``flashpolicies.policies``, defining acceptable values for\n        this argument.\n\n    **Optional arguments:**\n\n    ``domains``\n        A list of domains from which to allow access. Each value may\n        be either a domain name (e.g., ``example.com``) or a wildcard\n        (e.g., ``*.example.com``). Due to serious potential security\n        issues, it is strongly recommended that you not use wildcard\n        domain values.'

In [8]:
max_tok_length = 512

from transformers import AutoTokenizer

checkpoint = "Salesforce/codet5-large"

tokenizer = AutoTokenizer.from_pretrained(
    checkpoint,
    padding=True,
    pad_to_multiple_of=8,
    truncation=True,
    max_length=max_tok_length,
    )

The cache for model files in Transformers v4.22.0 has been updated. Migrating your old cache. This is a one-time only operation. You can interrupt this and resume the migration later on by calling `transformers.utils.move_cache()`.


0it [00:00, ?it/s]

tokenizer_config.json:   0%|          | 0.00/1.30k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/511k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/294k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.37M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/11.3k [00:00<?, ?B/s]

In [9]:
def preprocess_function(sample):
    model_inputs = tokenizer(
        sample["code"],
        text_target = sample["docstring"],
        )
    return model_inputs

In [10]:
test_dataset_filtered = test_dataset.filter(lambda x: len(preprocess_function(x)["input_ids"]) <= max_tok_length and len(preprocess_function(x)["labels"]) <= max_tok_length,
                                               desc=f"Discarding source and target sentences with more than {max_tok_length} tokens")

Discarding source and target sentences with more than 512 tokens:   0%|          | 0/1000 [00:00<?, ? examples…

Token indices sequence length is longer than the specified maximum sequence length for this model (877 > 512). Running this sequence through the model will result in indexing errors


In [11]:
print("Test dataset size:", len(test_dataset_filtered))

Test dataset size: 898


In [12]:
tokenized_test_dataset = test_dataset_filtered.map(preprocess_function, batched=True)

Map:   0%|          | 0/898 [00:00<?, ? examples/s]

In [13]:
dic = {}
for sample in tokenized_test_dataset:
    sample_length = len(sample['input_ids'])
    if sample_length not in dic:
        dic[sample_length] = 1
    else:
        dic[sample_length] += 1

for i in range(1,max_tok_length+1):
    if i in dic:
        print(f"{i:>2} {dic[i]:>3}")

40   1
42   1
44   2
45   1
46   3
48   4
49   3
50   3
52   6
53   1
54   2
55   3
56   2
57   2
58   4
59   5
60   3
61   5
62   4
63   3
64   3
65   4
66   4
67   7
68   4
69   3
70   3
71   4
72   3
73   6
74   7
75   8
78   2
79   5
80   4
81   4
82   5
83   3
84   5
85   5
86   2
87   4
88  11
89   6
90   7
91   3
92   8
93   3
94   3
95   4
96   9
97   3
98   5
99   5
100   4
101   4
102   2
103   5
104   3
105   2
106   5
107   4
108   4
109   7
110   2
111   5
112   1
113  11
114   4
115   4
116   6
117   3
118   4
119   2
120   5
121   3
122   3
123   4
124   3
125   8
126   4
127   2
128   1
129   4
130   6
131   3
132   4
133   3
134   4
135   4
136   1
137   5
138   5
139   2
140   4
141   5
142   3
143   2
144   3
145   4
146   3
147   5
148  10
149   3
150   4
151   1
152   6
153   2
154   4
155   5
156   4
157   5
158   5
159   1
160   3
161   3
162   6
163   5
164   1
165   1
166   3
167   2
168   2
169   1
170   1
171   2
172   3
173   2
175   5
176   4
177   5
178   

In [14]:
for sample in tokenized_test_dataset.select(range(3)):
    print(sample['input_ids'])
    print(sample['attention_mask'])
    print(sample['labels'])

[1, 536, 1361, 67, 7039, 12, 2890, 16, 1026, 67, 7039, 16, 2870, 67, 7039, 33, 7036, 4672, 203, 3639, 3536, 3116, 392, 1566, 3880, 12123, 203, 3639, 3422, 273, 1026, 67, 7039, 18, 3530, 1435, 203, 3639, 309, 2870, 67, 7039, 353, 486, 599, 30, 203, 5411, 3422, 18, 2725, 12, 7763, 67, 7039, 13, 203, 3639, 327, 3422, 2]
[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
[1, 3116, 392, 1566, 3880, 18, 2]
[1, 536, 855, 67, 3001, 12, 2890, 16, 276, 30, 609, 13, 317, 1426, 30, 203, 3639, 3536, 203, 3639, 20418, 326, 276, 910, 1160, 16, 5504, 783, 770, 471, 327, 1053, 203, 3639, 469, 327, 1083, 18, 2597, 999, 8032, 3001, 471, 518, 1807, 326, 1967, 487, 875, 316, 605, 26473, 18, 203, 3639, 3536, 203, 3639, 309, 365, 18, 896, 67, 9339, 13332, 203, 5411, 327, 1083, 203, 3639, 365, 6315, 3256, 18, 5688, 67, 2472, 1435, 203, 3639, 309, 276, 422, 365, 6315, 3256, 18, 3

In [15]:
import torch
from transformers import BitsAndBytesConfig

quantization_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_use_double_quant=True,
    bnb_4bit_compute_dtype=torch.bfloat16,
)

In [16]:
from transformers import AutoModelForSeq2SeqLM

model = AutoModelForSeq2SeqLM.from_pretrained(
    checkpoint,
    quantization_config=quantization_config
    )


config.json:   0%|          | 0.00/1.38k [00:00<?, ?B/s]

`low_cpu_mem_usage` was None, now default to True since model is quantized.


pytorch_model.bin:   0%|          | 0.00/1.48G [00:00<?, ?B/s]

### **Evaluación**

In [17]:
test_batch_size = 32
batch_tokenized_test = tokenized_test_dataset.batch(test_batch_size)

Batching examples:   0%|          | 0/898 [00:00<?, ? examples/s]

In [18]:
number_of_batches = len(batch_tokenized_test["code"])
output_sequences = []
sources = []

for i in range(number_of_batches):

    inputs = tokenizer(
        batch_tokenized_test["code"][i],
        max_length=max_tok_length,
        truncation=True,
        return_tensors="pt",
        padding=True,
    )
    output_batch = model.generate(
        input_ids=inputs["input_ids"].cuda(),
        attention_mask=inputs["attention_mask"].cuda(),
        max_length=max_tok_length,
        num_beams=1,
        do_sample=False
        )
    output_sequences.extend(output_batch.cpu())

In [19]:
decoded_preds = tokenizer.batch_decode(output_sequences, skip_special_tokens=True)

In [20]:
references = tokenizer.batch_decode(tokenized_test_dataset["labels"], skip_special_tokens=True)

In [21]:
sources = test_dataset_filtered['code']

In [22]:
decoded_preds[:1]

['(self, model, instance, extra_attrs=None):\n        """Build an attribute dictionary."""\n       ,,_,,):\n        = self.build_attrs(model, extra_attrs)\n        return attrs']

In [23]:
references[:1]

['Build an attribute dictionary.']

In [24]:
sources[:1]

['def build_attrs(self, base_attrs, extra_attrs=None):\n        """Build an attribute dictionary."""\n        attrs = base_attrs.copy()\n        if extra_attrs is not None:\n            attrs.update(extra_attrs)\n        return attrs']

In [25]:
from whisper.normalizers.basic import BasicTextNormalizer

normalizer = BasicTextNormalizer()

decoded_preds_clean = [normalizer(text) for text in decoded_preds]
references_clean = [normalizer(text) for text in references]
sources_clean = [normalizer(text) for text in sources]

In [26]:
from evaluate import load

metric = load("sacrebleu")

In [27]:
result = metric.compute(predictions=decoded_preds_clean, references=references_clean)
print(f'BLEU score: {result["score"]:.1f}')

BLEU score: 3.0


In [28]:
from evaluate import load

rouge_metric = load("rouge")
comet_metric = load("comet")

Fetching 5 files:   0%|          | 0/5 [00:00<?, ?it/s]

model.ckpt:   0%|          | 0.00/2.32G [00:00<?, ?B/s]

.gitattributes:   0%|          | 0.00/1.48k [00:00<?, ?B/s]

hparams.yaml:   0%|          | 0.00/567 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/3.38k [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.10M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/616 [00:00<?, ?B/s]

/usr/local/lib/python3.11/dist-packages/pytorch_lightning/core/saving.py:195: Found keys that are not in the model state dict but in the checkpoint: ['encoder.model.embeddings.position_ids']


In [29]:
comet_score = comet_metric.compute(predictions = decoded_preds_clean, references = references_clean, sources = sources_clean)

INFO:pytorch_lightning.utilities.rank_zero:GPU available: True (cuda), used: True
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs
INFO:pytorch_lightning.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


In [30]:
print(f"COMET: {comet_score['mean_score'] * 100:.2f} %")

COMET: 47.53 %


In [31]:
rouge_score = rouge_metric.compute(predictions = decoded_preds_clean, references = references_clean)

In [32]:
print(f"Rouge 1: {rouge_score['rouge1'] * 100:.2f} %")
print(f"Rouge 2: {rouge_score['rouge2'] * 100:.2f} %")
print(f"Rouge L: {rouge_score['rougeL'] * 100:.2f} %")

Rouge 1: 20.83 %
Rouge 2: 12.53 %
Rouge L: 19.24 %
